In [12]:
!pip install llama_index pyvis Ipython langchain pypdf -q
!pip install llama-index-llms-huggingface -q
!pip install llama-index-embeddings-langchain -q

In [13]:
import logging
import sys
#
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [14]:
!pip install langchain-community -q

In [15]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.llms.huggingface import HuggingFaceInferenceAPI
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from pyvis.network import Network
from langchain_community.embeddings.huggingface import HuggingFaceInferenceAPIEmbeddings

In [ ]:
HF_TOKEN = ""
llm = HuggingFaceInferenceAPI(
    model_name="HuggingFaceH4/zephyr-7b-beta", token=HF_TOKEN
) 

In [17]:
embed_model = LangchainEmbedding(
  HuggingFaceInferenceAPIEmbeddings(api_key=HF_TOKEN,model_name="thenlper/gte-large")
)

In [ ]:
documents = SimpleDirectoryReader("/content/Documents").load_data()
print(len(documents))

In [ ]:
from huggingface_hub import InferenceApi

token = HF_TOKEN
inference = InferenceApi(repo_id="HuggingFaceH4/zephyr-7b-beta", token=token)


In [ ]:
import requests

headers = {"Authorization": f"Bearer {token}"}
url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    print(response.json())
except requests.exceptions.HTTPError as e:
    print(f"HTTP Error: {e}")
    print(f"Response Content: {response.content}")


In [29]:
#setup the service context (global setting of LLM)
Settings.llm = llm
Settings.chunk_size = 512

#setup the storage context
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

#Construct the Knowlege Graph Undex
index = KnowledgeGraphIndex.from_documents(documents=documents,
                                           max_triplets_per_chunk=3,
                                           storage_context=storage_context,
                                           embed_model=embed_model,
                                           include_embeddings=True)

In [ ]:
query = "In the text, who said - 'cloudless sky'? "
query_engine = index.as_query_engine(include_text=True,
                                     response_mode ="tree_summarize",
                                     embedding_mode="hybrid",
                                     similarity_top_k=5,)
#
message_template =f"""<|system|>Please check if the following pieces of context has any mention of the  keywords provided in the Question.If not then don't know the answer, just say that you don't know.Stop there.Please donot try to make up an answer.</s>
<|user|>
Question: {query}
Helpful Answer:
</s>"""
#
response = query_engine.query(message_template)
#
print(response.response.split("<|assistant|>")[-1].strip())

In [ ]:
query = "In the text, who said - 'cloudless sky'? "
query_engine = index.as_query_engine(include_text=True,
                                     response_mode ="tree_summarize",
                                     embedding_mode="hybrid",
                                     similarity_top_k=5,)
#
message_template =f"""<|system|>Please check if the following pieces of context has any mention of the  keywords provided in the Question.If not then don't know the answer, just say that you don't know.Stop there.Please donot try to make up an answer.</s>
<|user|>
Question: {query}
Helpful Answer:
</s>"""
#
response = query_engine.query(message_template)
#
print(response.response.split("<|assistant|>")[-1].strip())

In [ ]:
from pyvis.network import Network
from IPython.display import display
g = index.get_networkx_graph()
net = Network(notebook=True,cdn_resources="in_line",directed=True)
net.from_nx(g)
net.show("graph.html")
net.save_graph("Knowledge_graph.html")
#
import IPython
IPython.display.HTML(filename="/content/Knowledge_graph.html")

In [42]:
storage_context.persist()